In [17]:
import nltk
nltk.download("punkt")
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# Importing Libraries needed for Tensorflow processing
import tensorflow as tf   #version 1.13.2
import numpy as np
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
import tensorflow as tf
print(tf.__version__)


2.14.0


In [20]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download("punkt")
import json
import pickle


import numpy as np
import random
from keras.models import Sequential
from keras.layers import Activation,Dropout,Dense
from keras.optimizers import SGD
from keras.models import load_model

#chatbot gui
from termcolor import colored


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [21]:
ls

intents.json  sample_data/


In [22]:
words=[]
documents=[]
classes=[]
ignore_words=['?','!','.','“', '”','(', ')', ',','’']

data_file = open('intents.json',)
intents=json.load(data_file)

In [23]:
nltk.download('wordnet')

for intent in intents['intents']:
  for pattern in intent['patterns']:
    w=nltk.word_tokenize(pattern)
    words.extend(w)

    documents.append((w,intent['tag']))
    # add to our classes list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

#lemizer and lower each word and remove duplicates
words=[lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words=sorted(list(set(words)))

#sort classes
classes=sorted(list(set(classes)))

#decuments= pattern , intents
print(len(documents),"documents")
#classes= intents
print(len(classes),"classes",classes)
#words = each words , vocabulary
print(len(words),"unique lemmatized words",words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))


[nltk_data] Downloading package wordnet to /root/nltk_data...


247 documents
45 classes [' CIVIL hod', ' CSE about ', ' CSE lab', ' ECE lab', ' EEE hod', ' FDT lab', ' library ', 'ABOUT COLLEGE', 'ADMISSIONS', 'ANTI RAGGING', 'B.Tech branches', 'CHEMISTRY DEPARTMENT AND LAB', 'CIVIL about', 'CIVIL lab', 'CSE hod', 'ECE about', 'ECE hod', 'EEE about', 'EEE lab', 'ENGLISH DEPARTMENT AND ENGLISH LAB', 'FOOD TECHNOLOGY about', 'Fee structure for B.Tech', 'GAMES AND SPORTS', 'HOSTELS AND STAFF QUARTERS', 'HUMANITIES AND SOCIAL SCIENCE', 'JNTUACEK placements', 'JNTUACEK stands', 'MATHEMATICS DEPARTMENT', 'MECH hod', 'MECH lab', 'MECHANICAL about', 'NSS', 'OFFICES', 'PHYSICS DEPARTMENT AND LAB', 'Principal of JNTUA Kalikiri', 'Vice-Principal of JNTUA Kalikiri', 'WORKSHOPS', 'courses', 'goodbye', 'greeting', 'hours', 'library timings', 'location', 'thanks', 'working days']
137 unique lemmatized words ["'s", 'about', 'activity', 'address', 'admission', 'anti', 'anyone', 'are', 'at', 'auditorium', 'available', 'avilable', 'b', 'b.tech', 'been', 'branch', 'b

In [24]:

#create training data
training=[]

#create output  empty array
output_=[0]*len(classes)


#bag of words algorithm
for doc in documents:
    #intialize bag for each word
    bag=[]

    pattern_word=doc[0]

    #lemmatize and lower each word in pattern_word
    pattern_word=[lemmatizer.lemmatize(word.lower()) for word in pattern_word]

    for w in words:
      bag.append(1) if w in pattern_word else bag.append(0)

    #output
    output_row=list(output_)
    output_row[classes.index(doc[1])]=1

    training.append([bag,output_row])

#shuffel features and turn into array
random.shuffle(training)
training=np.array(training)

#train_x is input and train_y is output
train_x=list(training[:,0])
train_y=list(training[:,1])

print('training data created')

training data created


<ipython-input-24-b4f0f526b433>:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(training)


In [28]:
#building sequential model with 3 layers

model=Sequential()
#first layer with 128 neurons
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
#second layer with 64 neurons
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
#output layer
model.add(Dense(len(train_y[0]),activation='softmax'))

# stochastic gradient decsent
from tensorflow.keras.optimizers.legacy import SGD

s = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#compiling model
model.compile(loss='categorical_crossentropy',optimizer = s,metrics=['accuracy'])

#fitting model
hist=model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,verbose=1,validation_split=0.2)
#saving model
model.save('chatbot_model.h5',hist)

print('model created')


Train on 197 samples, validate on 50 samples
Epoch 1/200
197/197 [==============================] - 0s 1ms/sample - loss: 3.8223 - acc: 0.0203 - val_loss: 3.7705 - val_acc: 0.1000
Epoch 2/200
197/197 [==============================] - 0s 319us/sample - loss: 3.7549 - acc: 0.0457 - val_loss: 3.7340 - val_acc: 0.1000
Epoch 3/200
180/197 [==========================>...] - ETA: 0s - loss: 3.7112 - acc: 0.0611    

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


197/197 [==============================] - 0s 380us/sample - loss: 3.7060 - acc: 0.0711 - val_loss: 3.6745 - val_acc: 0.1200
Epoch 4/200
197/197 [==============================] - 0s 304us/sample - loss: 3.6377 - acc: 0.0863 - val_loss: 3.6005 - val_acc: 0.1400
Epoch 5/200
197/197 [==============================] - 0s 377us/sample - loss: 3.5144 - acc: 0.1269 - val_loss: 3.5069 - val_acc: 0.1600
Epoch 6/200
197/197 [==============================] - 0s 289us/sample - loss: 3.3714 - acc: 0.1320 - val_loss: 3.3476 - val_acc: 0.2200
Epoch 7/200
197/197 [==============================] - 0s 367us/sample - loss: 3.2047 - acc: 0.1675 - val_loss: 3.1514 - val_acc: 0.2000
Epoch 8/200
197/197 [==============================] - 0s 308us/sample - loss: 2.9937 - acc: 0.2741 - val_loss: 2.9616 - val_acc: 0.3200
Epoch 9/200
197/197 [==============================] - 0s 317us/sample - loss: 2.9138 - acc: 0.2335 - val_loss: 2.8227 - val_acc: 0.3600
Epoch 10/200
197/197 [==============================]

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [29]:

words=pickle.load(open("words.pkl","rb"))
classes=pickle.load(open("classes.pkl","rb"))


In [32]:
def clean_up_sentence(sentence):
  #toknize sentence
  sentence_words=nltk.word_tokenize(sentence)
  #lemmtaized and lower each word
  sentence_words=[lemmatizer.lemmatize(word.lower()) for word in sentence_words]

  return sentence_words

def bow(sentence,words,show_details=False):
  #cleaning up sentence
  sentence_words=clean_up_sentence(sentence)

  bag=[0]*len(words)

  for s in sentence_words:
    for i,w in enumerate(words):
      if w==s:
        # bag equal one if current word is in the vocabulary position
        bag[i]=1

        if show_details==True:
          print('Found in %s',w)

  return np.array(bag)


def predict_class(sentence,model):
  #filter prediction that below threshold
  p=bow(sentence,words,show_details=False)
  res=model.predict(np.array([p]))[0]

  ERROR_THRESHOLD=0.25

  results=[[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]

  #sort by strong probability
  results.sort(key=lambda x:x[1],reverse=True)

  return_list=[]
  for r in results:
      return_list.append({"intent":classes[r[0]],"probability":str(r[1])})


  return return_list

In [ ]:
def getResponse(ints,intents_json):
  tag=ints[0]['intent']
  list_intents=intents_json['intents']

  for i in list_intents:

    if (tag==i['tag']):
      result=random.choice(i['responses'])
      break

  return result


def chatbot_response(msg):
  ints=predict_class(msg,model)
  res=getResponse(ints,intents)

  return res













run()

WELCOME TO CHATBOT

HI
visit https://www.jntuacek.ac.in/ to know more details about college 
WHO IS HOD OF CSE
Dr. SHAIK NASEERA
WHERE IS JNTUACEK LOCATED
Our location is Madanapalli Road, Kalikiri
WHO IS PRINCIPAL
Dr. M. L. S DEVAKUMAR GARU
